# Collection of Functions within notebook to then port to specific .py files

Useful reference: http://mathesaurus.sourceforge.net/idl-numpy.html


## APOD3D Helping function

In [2]:
import numpy as np

def do_apod3d(nx, ny, nz, squared):

    if squared:
        apodx = np.sin(np.pi*(np.arange(0, nx, 1, dtype=float)+0.5)/nx) ** 2 
        apody = np.sin(np.pi*(np.arange(0, ny, 1, dtype=float)+0.5)/ny) ** 2 
        apodz = np.sin(np.pi*(np.arange(0, nz, 1, dtype=float)+0.5)/nz) ** 2

        # TODO: Figure out what is meant to happen with these vectors
        apodxy = np.multiply(apodx, apody)
        apodxy = np.concatenate((apodxy, [nx,ny,nz]))
        apodz2 = np.transpose(np.concatenate((apodz, [nz, nz, ny])))

        apodxyz = apodz2

    return apodxyz

# Test call -> I can't verify if it's correct on IDL
# do_apod3d(20,20,20, True)

## noise_gate

In [6]:
# Performs noise-gating of images


# INPUT - data cube

# OUTPUT - noise-gated data cube

# OPTIONAL INPUTS - gate - use gating (default is Weiner Filter)
#                   dct - use Discrete Fourier Transform (much slower)
#                   perc - percentage level of samples to calculate noise level, default 0.5 (median)
#                   gamma - level of SNR to use as a cut-off value, default is 1.5
#                   image_indepen_noise - calculation for removal of image independent additive noise
# TO DO - Parallelise -> WHERE?

import numpy as np

# Recommend using "Utils" Object to hold all relevant functions / image information
class ImageCube:
    def __init__(self, image, fourier_image):
        self.image = image
        self.fourier_image = fourier_image

    def _estimate_shot_noise(self):
        fac = np.sum(np.sqrt(self.image[self.image>0]))
        betas = np.abs(self.fourier_image)/fac
        self.betas = betas
        return betas

    def _image_indepen_noise(self):
        betas = np.abs(self.fourier_image)
        self.betas = betas
        return betas

    def gate(self, gamm=1.5):
        filt = (1/gamm) * np.abs(self.fourier_image) / self.betas
        filt = filt[filt>1]
        return filt

    def wiener_filt(self, gamm=1.5):
        filt = np.abs(self.fourier_image) / (gamm*self.betas+np.abs(self.fourier_image))
        return filt


def noise_gate(data, gated_data=False, dct=False, image_indepen_noise=True):
    """
    Noise-gate a data cube.

    :param data: Data cube
    :param gated_data: ???
    :param dct: ???
    """
    sz = data.shape
    nx, ny, nz = sz[0], sz[1], sz[2]

    win_size = 12
    half_win = win_size / 2

    # Apodisation cube for sub-images
    apod = do_apod3d(win_size, win_size, win_size, squared=True)

    noise_elem = int(nx*ny / win_size / win_size)
    noise_arr = np.zeros(shape=(win_size, win_size, win_size, noise_elem))  # Scratch array = Zeroes array 

    # Loop for calculating noise profile -> Is this what can be parallelised?
    h=0  # Height in image cube ???
    print("Calculating noise model...")

    for i in np.arange(half_win, nx-half_win-1, step=win_size):
        for j in np.arange(half_win, ny-half_win-1, step=win_size):

            x = (int(i-half_win), int(i+half_win-1))
            y = (int(j-half_win), int(j+half_win-1))

            # SubImage and Apod
            subImage = data[x[0]:x[1], y[0]:y[1],0:win_size-1] 
            # subImage = subImage * apod   # ISSUES HERE - PROBABLY TO DO WITH APOD3D CODE
            print(subImage.shape, apod.shape)

            # if dct:
            #     fourier_image = dct_fft_3d(subImage, ortho=True)
            # else:
            #     fourier_image = None
            #     # TODO: ADD CASE WHERE PERFORMING FFT

            # subImageCube = ImageCube(subImage, fourier_image)

            # if image_indepen_noise:
            #     subImageCube._image_indepen_noise()
            # else:
            #     subImageCube._estimate_shot_noise()

            # noise_arr[:,:,:, h] = subImageCube.betas
            # h += 1

    return noise_arr

    # Set noise profile

if __name__ == '__main__':

    data_cube = np.random.rand(100,100,100)

    noise_arr = noise_gate(data=data_cube)

Calculating noise model...
(11, 11, 11) (15,)


TypeError: bad operand type for abs(): 'NoneType'

In [ ]:
noise_arr